# Deciphering the Flapping Frequency Allometry: Unveiling the Role of Sustained Body Attitude in the Aerodynamic Scaling of Normal Hovering Animals

### Jeremy Pohly<sup>1</sup>, Chang-kwon Kang<sup>1</sup> and Hikaru Aono<sup>2</sup>

1. Department of Mechanical and Aerospace Engineering, University of Alabama in Huntsville, Huntsville, Alabama 35899, United States of America
2. Department of Mechanical Engineering and Robotics, Shinshu University, Ueda, Nagano 3868567, Japan


In [ ]:
# Load libararies
library(latex2exp)
library(ggplot2)

library(phytools) 
library(ape)
library(geiger)

library(nlme) 
library("wesanderson")


In [ ]:
# Data file
data_file = './HoverScaling_Bio_ESM_DataS1.csv'
data_file_stripped = gsub("\\.csv", "", data_file)

# Choose if the considered data is extended or base 
suppl_bool = FALSE  # TRUE: extended data; FALSE: base data

# Save locations
results_path = './results/'
if (!dir.exists(results_path)) {
    dir.create(results_path)
}

if (suppl_bool) {
    save_path = paste0(results_path, 'supplementary/')
} else {
    save_path = paste0(results_path, 'base/')
}

figures_save_path = paste0(save_path, 'figures/')
if (!dir.exists(save_path)) {
    dir.create(save_path)
}
if (!dir.exists(figures_save_path)) {
    dir.create(figures_save_path)
}



In [ ]:
# Load data
data <- read.csv(data_file)
df = data.frame(data)

# Replace Amazilia_fimbriata fluviatilis with Amazilia_fimbriata as it is a subspecies
df$species<-gsub("fimbriata fluviatilis", "fimbriata", df$species)


In [ ]:
# if we consider the base data, remove the extended data
extended_researchers <- list("Norberg et al.", "Hakansson et al.", "Weis-Fogh")

if (suppl_bool) {
    cat("Extended data")
} else {
    cat("Base data")
    for (x in extended_researchers) {
       df <- df[!grepl(x, df$Researcher), (invert = TRUE), ]
    }
}


In [ ]:
# Construct the species name
df$Species <- paste(df$genus,df$species,sep="_")


In [ ]:
# Average the data over the individual speciesspecies
df_avg <- data.frame()

columns_list = c("mass...mg.", "freq...Hz.")
species_list <- unique(df$Species)

for (s in species_list){

    # Type
    df_type <- df[df$Species == s, c("Bird.Insect")][1]

    # Mass
    df_mass <- df[df$Species == s, c("species", "N", "mass...mg.")]
    N_mass <- na.omit(df_mass)["N"]
    avg_mass <- weighted.mean(df_mass["mass...mg."], df_mass["N"], na.rm=TRUE)

    # Wing length
    df_wing_length <- df[df$Species == s, c("species", "N", "wing.length...mm.")]
    N_wing_length <- na.omit(df_wing_length)["N"]
    avg_wing_length <- weighted.mean(df_wing_length["wing.length...mm."], df_wing_length["N"], na.rm=TRUE)

    # Body length
    df_body_length <- df[df$Species == s, c("species", "N", "body.length...mm.")]
    N_body_length <- na.omit(df_body_length)["N"]
    avg_body_length <- weighted.mean(df_body_length["body.length...mm."], df_body_length["N"], na.rm=TRUE)

    # Body attitude
    df_body_attitude <- df[df$Species == s, c("species", "N", "body.attitude...deg.")]
    N_body_attitude <- na.omit(df_body_attitude)["N"]
    avg_body_attitude <- weighted.mean(df_body_attitude["body.attitude...deg."], df_body_attitude["N"], na.rm=TRUE)

    # Wing area
    df_wing_area <- df[df$Species == s, c("species", "N", "wing.area..mm.2.")]
    N_wing_area <- na.omit(df_wing_area)["N"]
    avg_wing_area <- weighted.mean(df_wing_area["wing.area..mm.2."], df_wing_area["N"], na.rm=TRUE)

    # Frequency
    df_freq <- df[df$Species == s, c("species", "N", "freq...Hz.")]
    N_freq <- na.omit(df_freq)["N"]
    avg_freq <- weighted.mean(df_freq["freq...Hz."], df_freq["N"], na.rm=TRUE)

    # Amplitude
    df_amp <- df[df$Species == s, c("species", "N", "amp...deg.")]
    N_amp <- na.omit(df_amp)["N"]
    avg_amp <- weighted.mean(df_amp["amp...deg."], df_amp["N"], na.rm=TRUE)

    # Wing velocity
    df_Utip <- df[df$Species == s, c("species", "N", "U.tip..m.s.")]
    N_Utip <- na.omit(df_Utip)["N"]
    avg_Utip <- weighted.mean(df_Utip["U.tip..m.s."], df_Utip["N"], na.rm=TRUE)

    # CL
    df_CL <- df[df$Species == s, c("species", "N", "CL....")]
    N_CL <- na.omit(df_CL)["N"]
    avg_CL <- weighted.mean(df_CL["CL...."], df_CL["N"], na.rm=TRUE)
    
    # AR
    df_AR <- df[df$Species == s, c("species", "N", "AR....")]
    N_AR <- na.omit(df_AR)["N"]
    avg_AR <- weighted.mean(df_AR["AR...."], df_AR["N"], na.rm=TRUE)

    # Re
    df_Re <- df[df$Species == s, c("species", "N", "Re....")]
    N_Re <- na.omit(df_Re)["N"]
    avg_Re <- weighted.mean(df_Re["Re...."], df_Re["N"], na.rm=TRUE)

    # Reduced frequency
    df_red_freq <- df[df$Species == s, c("species", "N", "red..freq.....")]
    N_red_freq <- na.omit(df_red_freq)["N"]
    avg_red_freq <- weighted.mean(df_red_freq["red..freq....."], df_red_freq["N"], na.rm=TRUE)

    new_row <- data.frame(
        type = df_type[1],
        species = s, 
        N_mass = sum(N_mass), mass = avg_mass,
        N_body_length = sum(N_body_length), body_length = avg_body_length,
        N_body_attitude = sum(N_body_attitude), body_attitude = avg_body_attitude,
        N_wing_length = sum(N_wing_length), wing_length = avg_wing_length,
        N_wing_area = sum(N_wing_area), wing_area = avg_wing_area,
        N_freq = sum(N_freq), freq = avg_freq,
        N_amp = sum(N_amp), amp = avg_amp,
        N_Utip = sum(N_Utip), Utip = avg_Utip,
        N_AR = sum(N_AR), AR = avg_AR,
        N_CL = sum(N_CL), CL = avg_CL,
        N_red_freq = sum(N_red_freq), red_freq = avg_red_freq,
        N_Re = sum(N_Re), Re = avg_Re)
    df_avg <- rbind(df_avg, new_row)

}

# Sort by mass
df_avg <- df_avg[order(df_avg$mass),]


In [ ]:
# Load the phylogenetic tree

tree_filename <- "phyloT_complete_renamed.nwk"
treeSpecies <- read.tree(tree_filename)

rooted_treeSpecies = ape::compute.brlen(treeSpecies) # Branch length calculation


In [ ]:
# Create a dataframe with the species name as the index.
df_data <- df_avg[,2:ncol(df_avg)]
rownames(df_data) <- df_data$species 
df_data <- subset(df_data, select = -c(species))


In [ ]:
#Now, prune the tree to match the data and vice-versa:
data_matched<-treedata(phy=rooted_treeSpecies,data=df_data,sort=F,warnings=FALSE)


In [ ]:
power_fit_plot <- function (main_tree, x_in, y_in, species, type, 
                            ylim, xlabel, ylabel, h_just, info_bool, file_name, 
                            m_theory, breaks_array, grid_bool, log_scale=TRUE, 
                            sublabel, sublabel_x, sublabel_y, df_symbol) {        

  suppressWarnings({

  df_subdata <- data.frame(x_in, y_in, type, species)
  df_subdata <- na.omit(df_subdata)  
  rownames(df_subdata) <- df_subdata$species 

  # Only consider the matched data     
  data_matched <- treedata(phy=main_tree,data=df_subdata,sort=F,warnings=FALSE)    
  df_matched <- data.frame(data_matched$data)
  df_matched$x <- as.numeric(as.character(df_matched$x_in))   
  df_matched$y <- as.numeric(as.character(df_matched$y_in))  
  df_matched <- subset(df_matched, select = -c(x_in,y_in) )    
 
  x <- df_matched$x
  y <- df_matched$y    
      
  # OLS
  y_lm <- lm(log10(y) ~ log10(x))

  # OLS statistics
  m <- y_lm$coefficients[2]; names(m) <- NULL    
  b <- y_lm$coefficients[1]; names(b) <- NULL
  slope_ols <- m; names(slope_ols) <- NULL
  rsquared_value_ols <- summary(y_lm)$adj.r.squared; names(rsquared_value_ols) <- NULL
  conf_int_ols <- confint(y_lm)
  conf_int_slope_ols <- conf_int_ols["log10(x)", ]; names(conf_int_slope_ols) <- NULL
  intercept_ols <- y_lm$coefficients[1]; names(intercept_ols) <- NULL
  p_value_ols <- summary(y_lm)$coefficients[8]; names(p_value_ols) <- NULL
      
  # PGLS  
  spc<-rownames(data_matched$data)    
  V <-corPagel(value=0.5, phy=data_matched$phy, form = ~spc, fixed=TRUE)    
  C <- vcv.phylo(phy = data_matched$phy)

  df_matched$spc <- spc    
  y_pgls<-gls(log10(y)~log10(x), correlation = V, data=df_matched)        

  # PGLS statistics    
  m_pgls <- y_pgls$coefficients[2]
  b_pgls <- y_pgls$coefficients[1]
  slope_pgls <- y_pgls$coefficients[2]; names(slope_pgls) <- NULL
  conf_int_pgls <- confint(y_pgls)
  conf_int_slope_pgls <- conf_int_pgls["log10(x)", ]; names(conf_int_slope_pgls) <- NULL
  intercept_pgls <- y_pgls$coefficients[1];   names(intercept_pgls) <- NULL
  p_value_pgls <- summary(y_pgls)$tTable[2,"p-value"]; names(p_value_pgls) <- NULL 

  # trend line: OLS    
  x_base <- c(min(x), max(x))         
  line_ols_df <- data.frame(
    x_ols = x_base,
    y_ols = 10^b*c(min(x), max(x))^m
  )      

  # trend line: theory        
  line_df <- data.frame(
    x_theory = x_base,
    y_theory = 10^b*c(min(x), max(x))^m_theory
  )
      
  # trend lines: PGLS
  line_pgls_df <- data.frame(
    x_pgls = x_base,
    y_pgls = 10^b_pgls*c(min(x), max(x))^m_pgls
  )      

  # figure options    
  font_size <- 10
  yplot <- if (log_scale) log10(y) else y          
  ymax <- max(y) * 1.1
  ymin <- min(y) * 0.9    

  ind = unique(match(df_matched$type, df_symbol$species, nomatch = 0, incomparables = NULL))
  symbols_species = df_symbol$species[ind]    
  colors = df_symbol$colors[ind]        
  shapes = df_symbol$shapes[ind]    

  # figure    
  p <- ggplot(df_matched, aes(x = log10(x))) +
    stat_smooth(aes(y = if (log_scale) log10(y) else y), method = lm, size = 0.5, se = TRUE) +
    geom_point(aes(y = if (log_scale) log10(y) else y, color=df_matched$type, shape=df_matched$type), size=1) +     
    scale_color_manual(values=colors, breaks=symbols_species) +
    scale_shape_manual(values=shapes, breaks=symbols_species) +          
    scale_y_continuous(
      labels = if (log_scale) function(x) round(10^x, 2) else function(x) round(x, 2),
      breaks = if (log_scale) log10(breaks_array) else breaks_array,
      limits = if (log_scale) log10(c(ylim[1], ylim[2])) else c(ylim[1], ylim[2])
    ) +
    geom_line(data = line_ols_df, aes(x = log10(x_ols), y = if (log_scale) log10(y_ols) else y_ols), color = "blue", linetype = "solid") +       
    geom_line(data = line_pgls_df, aes(x = log10(x_pgls), y = if (log_scale) log10(y_pgls) else y_pgls), color = "green4", linetype = "solid") +                              
    xlab(xlabel) +
    ylab(ylabel) +
    theme(
      panel.background = element_blank(),
      panel.grid.major = if (grid_bool) element_line(color = "#dfdfdf") else element_blank(),
      panel.grid.minor = if (grid_bool) element_line(color = "#dfdfdf") else element_blank(),
      axis.line = element_line(color = "black"),
      plot.title = element_text(size = font_size),
      axis.title.x = element_text(size = font_size),
      axis.title.y = element_text(size = font_size),
      axis.text.x = element_text(size = font_size),
      axis.text.y = element_text(size = font_size),
      panel.spacing = unit(0.1, "lines"),
      plot.margin = unit(c(0.5, 0.5, 0.5, 0.5), "cm"),
      legend.text = element_text(size=4),
      legend.position="none",
      plot.caption = element_text(size = (font_size+2), hjust = -0.3, vjust = 7)
    ) +
    guides(color = guide_legend(ncol = 1)) +
    geom_line(data = line_df, aes(x = log10(x_theory), y = if (log_scale) log10(y_theory) else y_theory), color = "red", linetype = "dashed") +
    labs(caption = sublabel)                                
              
  })    
                
  # Save the figure as a pdf file
  file_name_pdf <- paste(file_name, ".pdf", sep="")
  width <- 2.25
  height <- 2.2
  suppressMessages(ggsave(file_name_pdf, width = width, height = height, dpi=300))
  cat("Saved ", file_name_pdf, "\n")

  return(list(slope_ols, intercept_ols, rsquared_value_ols, p_value_ols, conf_int_slope_ols, slope_pgls, intercept_pgls, p_value_pgls, conf_int_slope_pgls))

}

In [ ]:
# Define the symbols so that each symbol is uniquely associated with a type
species_names <- c("chalcid wasp", "vegetable leafminer", "fruit fly","mosquito", "cranefly", "crane fly",
                   "hoverfly", "ladybug", "green-veined white", "blow fly", "bicolored swallow", "common wasp",
                   "honey bee", "drone fly", "longhorn beetle", "large white", "bumblebee", "hummingbird hawk moth",
                   "summer chafer", "European rose chafer", "bumblebee (red-tailed)", "cockchafer", "european hornet",
                   "hawk moth", "privet hawk moth", "hummingbird", "dung beetle", "bat")                   

N_symbols <- length(species_names)
N_std_shape <- 15

df_symbol <- data.frame(
row.names = species_names,
species = species_names,
colors = wes_palette("Zissou1", N_symbols, type = "continuous"),    
shapes =  0:(N_symbols-1) %% N_std_shape # 
)


In [ ]:
# Function to store statistics
summary_data <- function(parameter_title,N_species,N_observations,stats) {
    summary_df <- data.frame(
        parameter = parameter_title,
        N_spec = N_species,
        N_obs = N_observations,
        gamma_ols = c(stats[[1]]),
        beta_ols = c(stats[[2]]),
        r2 = c(stats[[3]]),
        p_value = c(stats[[4]]),
        ci_95_lower = c(stats[[5]][1]),
        ci_95_upper = c(stats[[5]][2]),
        gamma_pgls = c(stats[[6]]),
        beta_pgls = c(stats[[7]]),
        p_value_pgls = c(stats[[8]]),
        ci_95_lower_pgls = c(stats[[9]][1]),
        ci_95_upper_pgls = c(stats[[9]][2]),
        stringsAsFactors = FALSE           
    )
    return(summary_df)
}

In [ ]:
# Subfigure caption labels
if (suppl_bool) { # supplementary
    sublabel_body_length = "D"
    sublabel_body_attitude = "E"
    sublabel_wing_length = "A"
    sublabel_wing_area = "B"
    sublabel_freq = "F"
    sublabel_amp = "G"
    sublabel_AR = "C"
    sublabel_Utip = "H"
    sublabel_CL = "J"
    sublabel_Re = "K"
    sublabel_red_freq = "I"
} else { # base
    sublabel_body_length = "E"
    sublabel_body_attitude = "F"
    sublabel_wing_length = "B"
    sublabel_wing_area = "C"
    sublabel_freq = "G"
    sublabel_amp = "H"
    sublabel_AR = "D"
    sublabel_Utip = "I"
    sublabel_CL = "K"
    sublabel_Re = "L"
    sublabel_red_freq = "J"
}

info_bool <- FALSE
grid_bool <- FALSE
sublabel_x = -4
sublabel_y = 0

# Perform the regression analysis and generate the figures
# Span
fig_name = "fig_span_vs_mass"
fig_save_name = paste(figures_save_path, fig_name, sep="")
ylim = c(0.5, 200)
xlabel = TeX("$log_{10}mass \\ [mg]$")
ylabel = TeX("Wing Length [mm]")
m_theory = 1/3	
y_ticks = c(1,10, 100)
stats_span = power_fit_plot(rooted_treeSpecies, df_avg$mass, df_avg$wing_length, df_avg$species, df_avg$type,
  ylim, xlabel, ylabel, 1, info_bool, fig_save_name, m_theory, y_ticks, grid_bool, log_scale=TRUE, sublabel_wing_length, sublabel_x, sublabel_y, df_symbol)

# Area 
fig_name = "fig_area_vs_mass"
fig_save_name = paste(figures_save_path, fig_name, sep="")
ylim = c(0.1, 10000)
xlabel = TeX("$log_{10}mass \\ [mg]$")
ylabel = TeX("Wing Area $[mm^2]$")
m_theory = 2/3
y_ticks = c(0.1, 10, 1000)
stats_area = power_fit_plot(rooted_treeSpecies, df_avg$mass, df_avg$wing_area, df_avg$species, df_avg$type,
  ylim, xlabel, ylabel, 1, info_bool, fig_save_name, m_theory, y_ticks, grid_bool, log_scale=TRUE, sublabel_wing_area, sublabel_x, sublabel_y, df_symbol)

# Aspect Ratio
fig_name = "fig_AR_vs_mass"
fig_save_name = paste(figures_save_path, fig_name, sep="")
# ylim = c(0., 2.)
ylim = c(0.5, 10)
xlabel = TeX("$log_{10}mass \\ [mg]$")
ylabel = TeX("Aspect Ratio")
# y_ticks = c(3, 4, 5)
y_ticks = c(0.5, 1, 2, 4, 8)
m_theory = 0.0
stats_AR = power_fit_plot(rooted_treeSpecies, df_avg$mass, df_avg$AR, df_avg$species, df_avg$type,
  ylim, xlabel, ylabel, 1, info_bool, fig_save_name, m_theory, y_ticks, grid_bool, log_scale=TRUE, sublabel_AR, sublabel_x, sublabel_y, df_symbol)

# Body length
fig_name = "fig_body_length_vs_mass"
fig_save_name = paste(figures_save_path, fig_name, sep="")
ylim = c(3, 100)
xlabel = TeX("$log_{10}mass \\ [mg]$")
ylabel = TeX("Body Length [mm]")
m_theory = 1/3
y_ticks = c(5, 10, 20, 40, 80)
stats_body_length = power_fit_plot(rooted_treeSpecies, df_avg$mass, df_avg$body_length, df_avg$species, df_avg$type,
  ylim, xlabel, ylabel, 1, info_bool, fig_save_name, m_theory, y_ticks, grid_bool, log_scale=TRUE, sublabel_body_length, sublabel_x, sublabel_y, df_symbol)

# Body attitude
fig_name = "fig_body_attitude_vs_mass"
fig_save_name = paste(figures_save_path, fig_name, sep="")
#ylim = c(0.0, 90)
ylim = c(10, 100)
xlabel = TeX("$log_{10}mass \\ [mg]$")
ylabel = TeX("Body Attitude [deg]")
m_theory = 0.0
y_ticks = c(10, 20, 40, 80)
stats_body_attitude = power_fit_plot(rooted_treeSpecies, df_avg$mass, df_avg$body_attitude, df_avg$species, df_avg$type,
  ylim, xlabel, ylabel, 1, info_bool, fig_save_name, m_theory, y_ticks, grid_bool, log_scale=TRUE, sublabel_body_attitude, sublabel_x, sublabel_y, df_symbol)

# Frequency
fig_name = "fig_freq_vs_mass"
fig_save_name = paste(figures_save_path, fig_name, sep="")
# ylim = c(10.0, 450.)
ylim = c(5, 1000)
xlabel = TeX("$log_{10}mass \\ [mg]$")
ylabel = TeX("Flapping Frequency [Hz]")
# y_ticks = c(1.0, 125, 250)
y_ticks = c(10, 40, 160, 640)
m_theory = -1/6
stats_freq = power_fit_plot(rooted_treeSpecies, df_avg$mass, df_avg$freq, df_avg$species, df_avg$type,
  ylim, xlabel, ylabel, 1, info_bool, fig_save_name, m_theory, y_ticks, grid_bool, log_scale=TRUE, sublabel_freq, sublabel_x, sublabel_y, df_symbol)

# Amplitude 
fig_name = "fig_amp_vs_mass"
fig_save_name = paste(figures_save_path, fig_name, sep="")
ylim = c(35, 210.0)
xlabel = TeX("$log_{10}mass \\ [mg]$")
ylabel = TeX("Flapping Amplitude [deg]")
sublabel_y = -30
m_theory = 0.0
y_ticks = c(45, 90, 180)
stats_amp = power_fit_plot(rooted_treeSpecies, df_avg$mass, df_avg$amp, df_avg$species, df_avg$type,
  ylim, xlabel, ylabel, 1, info_bool, fig_save_name, m_theory, y_ticks, grid_bool, log_scale=TRUE, sublabel_amp, sublabel_x, sublabel_y, df_symbol)

# Velocity
fig_name = "fig_vel_vs_mass"
fig_save_name = paste(figures_save_path, fig_name, sep="")
ylim = c(0.5, 20)
xlabel = TeX("$log_{10}mass \\ [mg]$")
ylabel = TeX("Wing Velocity [m/s]")
m_theory = 1/6
y_ticks = c(1, 2, 4, 8, 16)
stats_vel = power_fit_plot(rooted_treeSpecies, df_avg$mass, df_avg$Utip, df_avg$species, df_avg$type,
  ylim, xlabel, ylabel, 1, info_bool, fig_save_name, m_theory, y_ticks, grid_bool, log_scale=TRUE, sublabel_Utip, sublabel_x, sublabel_y, df_symbol)

# Reduced frequency
fig_name = "fig_k_vs_mass"
fig_save_name = paste(figures_save_path, fig_name, sep="")
ylim = c(0.05, 1.5)
xlabel = TeX("$log_{10}mass \\ [mg]$")
ylabel = TeX("Reduced Frequency")
m_theory = 0.0
y_ticks = c(0.05,  0.2,  0.8)
stats_k = power_fit_plot(rooted_treeSpecies, df_avg$mass, df_avg$red_freq, df_avg$species, df_avg$type,
  ylim, xlabel, ylabel, 1, info_bool, fig_save_name, m_theory, y_ticks, grid_bool, log_scale=TRUE, sublabel_red_freq, sublabel_x, sublabel_y, df_symbol)

# CL
fig_name = "fig_CL_vs_mass"
fig_save_name = paste(figures_save_path, fig_name, sep="")
#ylim = c(0.0, 2.0)
ylim = c(0.1, 4)
xlabel = TeX("$log_{10}mass \\ [mg]$")
ylabel = TeX("Lift Coefficient")
m_theory = 0.0
#y_ticks = c(0, 1.0, 2.0)
y_ticks = c(0.25, 1, 4)
stats_CL = power_fit_plot(rooted_treeSpecies, df_avg$mass, df_avg$CL, df_avg$species, df_avg$type,
  ylim, xlabel, ylabel, 1, info_bool, fig_save_name, m_theory, y_ticks, grid_bool, log_scale=TRUE, sublabel_CL, sublabel_x, sublabel_y, df_symbol)

# Reynolds number
fig_name = "fig_Re_vs_mass"
fig_save_name = paste(figures_save_path, fig_name, sep="")
ylim = c(10, 41000)
xlabel = TeX("$log_{10}mass \\ [mg]$")
ylabel = TeX("Reynolds Number")
y_ticks = c(15, 150, 1500, 15000)
m_theory = 0.5
stats_Re = power_fit_plot(rooted_treeSpecies, df_avg$mass, df_avg$Re, df_avg$species, df_avg$type,
  ylim, xlabel, ylabel, 1, info_bool, fig_save_name, m_theory, y_ticks, grid_bool, log_scale=TRUE, sublabel_Re, sublabel_x, sublabel_y, df_symbol)


# Create a dataframe of the resulting statistics
summary_df <- summary_data(c("Span"),sum(!is.na(df_avg$wing_length)),sum(df_avg$N_wing_length),stats_span)
summary_df <- rbind(summary_df, summary_data(c("Area"),sum(!is.na(df_avg$wing_area)),sum(df_avg$N_wing_area),stats_area))
summary_df <- rbind(summary_df, summary_data(c("Aspect Ratio"),sum(!is.na(df_avg$AR)),sum(df_avg$N_AR),stats_AR))
summary_df <- rbind(summary_df, summary_data(c("Body Length"),sum(!is.na(df_avg$body_length)),sum(df_avg$N_body_length),stats_body_length))
summary_df <- rbind(summary_df, summary_data(c("Body Attitude"),sum(!is.na(df_avg$body_attitude)),sum(df_avg$N_body_attitude),stats_body_attitude))
summary_df <- rbind(summary_df, summary_data(c("Frequency"),sum(!is.na(df_avg$freq)),sum(df_avg$N_freq),stats_freq))
summary_df <- rbind(summary_df, summary_data(c("Amplitude"),sum(!is.na(df_avg$amp)),sum(df_avg$N_amp),stats_amp))
summary_df <- rbind(summary_df, summary_data(c("Velocity"),sum(!is.na(df_avg$Utip)),sum(df_avg$N_Utip),stats_vel))
summary_df <- rbind(summary_df, summary_data(c("Reduced Frequency"),sum(!is.na(df_avg$red_freq)),sum(df_avg$N_red_freq),stats_k))
summary_df <- rbind(summary_df, summary_data(c("CL"),sum(!is.na(df_avg$CL)),sum(df_avg$N_CL),stats_CL))
summary_df <- rbind(summary_df, summary_data(c("Reynolds Number"),sum(!is.na(df_avg$Re)),sum(df_avg$N_Re),stats_Re))

writeLines("\n------------------\n")
cat("Summary Statistics:")
View(summary_df)

# Save the dataframe to a csv file
stats_file_path = paste0(save_path, 'stats_summary.csv')
write.csv(summary_df, file = stats_file_path, row.names = FALSE)
writeLines("\n------------------\n")
cat("Saved ", stats_file_path, "\n")